In [1]:
!pip3 install -q -U bitsandbytes peft trl accelerate datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 48.2 MB/s eta 0:00:00


In [2]:
import os
import torch
import transformers
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GemmaTokenizer

In [3]:
os.environ["HF_Token"]=userdata.get("HF_Token")

In [4]:
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
model_name="google/gemma-2b"

In [6]:
tokenizer=AutoTokenizer.from_pretrained(model_name, token=os.environ["HF_Token"])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
model=AutoModelForCausalLM.from_pretrained(model_name,
                                           quantization_config=bnb_config,
                                           device_map={"":0},
                                           token=os.environ["HF_Token"])

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
text="Quote: Imagination is more,"
# pick device automatically
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs=tokenizer(text, return_tensors="pt").to(device)
outputs=model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto, Portugal.

I have always been fascinated by the power of imagination and the way it can transform our reality.

I


In [9]:
text="Quote: Imagination is more,"
# pick device automatically
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs=tokenizer(text, return_tensors="pt").to(device)
outputs=model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in the beautiful city of Porto, Portugal.

I have always been fascinated by the power of imagination and the way it can transform our reality.

I


In [10]:
os.environ["WANDB_DISABLED"]="false"

In [11]:
from peft import TaskType

In [12]:
lora_config=LoraConfig(
    r=8,
    target_modules=['q_proj', 'o_proj', 'k_proj', 'v_proj', 'gate_proj', ' up_proj', 'down_proj'],
    task_type=TaskType.CAUSAL_LM
)

In [13]:
dataset=load_dataset("Abirate/english_quotes")

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [15]:
data=dataset.map(lambda samples: tokenizer(samples['quote']),batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [16]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [17]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [18]:
len(data['train'])

2508

In [19]:
def formatting_func(example):
  text=f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
  return [text]

In [20]:
formatting_func(data['train'])

['Quote: “Be yourself; everyone else is already taken.”\nAuthor: Oscar Wilde']

In [21]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [22]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `finetuned_model` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authe

In [23]:
!hf auth whoami

A-Asif


In [24]:
trainer=SFTTrainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        optim='paged_adamw_8bit'
    ),
    peft_config=lora_config,
    formatting_func=formatting_func
)

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:687: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [25]:
trainer

In [26]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: as29624041 (as29624041-university-of-engineering-and-technology-taxila) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.561500
2,1.626700
3,2.482600
4,2.756100
5,2.300200
6,2.480800
7,2.887100
8,2.251900
9,3.193900
10,2.220800


TrainOutput(global_step=100, training_loss=2.060578085184097, metrics={'train_runtime': 181.7204, 'train_samples_per_second': 2.201, 'train_steps_per_second': 0.55, 'total_flos': 189491484696576.0, 'train_loss': 2.060578085184097})

In [27]:
new_model="gemma-2b-finetuned-with-Abirate-english_quotes"
# Save trained model
trainer.model.save_pretrained(new_model)

In [28]:
from peft import PeftModel

In [29]:
model = PeftModel.from_pretrained(model, new_model)
model = model.merge_and_unload()

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [30]:
text="Quote: Two things are infinite: the universe and human stupidity;"
# pick device automatically
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs=tokenizer(text, return_tensors="pt").to(device)
outputs=model.generate(**inputs, max_new_tokens=20)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.
- Albert Einstein

I'm not sure


In [31]:
text="Quote: Be who you are and say what you feel, because those who mind don't matter,"
# pick device automatically
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs=tokenizer(text, return_tensors="pt").to(device)
outputs=model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.

I am a very private person and I don't like to talk about myself. I am a very private person and I don't like to talk about myself. I am a very private person


In [32]:
# load llama tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [33]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [34]:
pip install --upgrade huggingface_hub

In [35]:
model.push_to_hub("A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes", check_pr=True,use_auth_token=True)

/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...te-english_quotes/model.safetensors:   1%|1         | 33.3MB / 3.21GB            

CommitInfo(commit_url='https://huggingface.co/A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes/commit/74b2a679d44c50d4682c235624b3da36aca7efd0', commit_message='Upload GemmaForCausalLM', commit_description='', oid='74b2a679d44c50d4682c235624b3da36aca7efd0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes', endpoint='https://huggingface.co', repo_type='model', repo_id='A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes'), pr_revision=None, pr_num=None)

In [36]:
tokenizer.push_to_hub("A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes",check_pr=True,use_auth_token=True)

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...rate-english_quotes/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...irate-english_quotes/tokenizer.json:  94%|#########4| 32.4MB / 34.4MB            

CommitInfo(commit_url='https://huggingface.co/A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes/commit/a6c419be2c62340f31f7fe22c4ba8ecfd9fb0fab', commit_message='Upload tokenizer', commit_description='', oid='a6c419be2c62340f31f7fe22c4ba8ecfd9fb0fab', pr_url=None, repo_url=RepoUrl('https://huggingface.co/A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes', endpoint='https://huggingface.co', repo_type='model', repo_id='A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes'), pr_revision=None, pr_num=None)

In [37]:
model_name="A-Asif/gemma-2b-finetuned-with-Abirate-english_quotes"
model=AutoModelForCausalLM.from_pretrained(model_name,
                                           quantization_config=bnb_config,
                                           device_map={"":0},
                                           token=os.environ["HF_Token"])

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:231: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/3.21G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [39]:
model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
      )
    )
    (n

In [38]:
# load llama tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

In [40]:
text="Quote: Be who you are and say what you feel, because those who mind don't matter,"
# pick device automatically
device = "cuda" if torch.cuda.is_available() else "cpu"

inputs=tokenizer(text, return_tensors="pt").to(device)
outputs=model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Be who you are and say what you feel, because those who mind don't matter, and those who matter don't mind.

I am a very private person and I don't like to talk about myself. I am a very private person and I don't like to talk about myself. I am a very private person
